In [3]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.5 MB/s eta 0:00:00


In [4]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.1 MB/s eta 0:00:00


In [5]:
!pip install datasets
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install nlpaug
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-xhh0rwtm
  Running command git clone --filter=blob:none --quiet https://github

In [6]:

import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import os, sys, itertools, re

from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertModel, DistilBertConfig
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split

import nlpaug.augmenter.word as naw

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset as d1,load_dataset, load_from_disk, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, AdamW
import contractions
from transformers import (ElectraForSequenceClassification,
                          ElectraTokenizerFast, EvalPrediction, InputFeatures,
                          Trainer, TrainingArguments, glue_compute_metrics)
from torch.utils.data import Dataset




In [7]:
# List of unique accessions
unique_accessions = ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']



# Dictionary to store the dataframes
dfs = {}

# Loop through the unique accessions
for accession in unique_accessions:
    # Create the dataframe name
    path = '/content/drive/MyDrive/NAEP_Comp/'
    df_name = 'df_cleaned' + accession

    # Read the CSV file into a dataframe
    df = pd.read_csv(path + df_name + '.csv')

    # Add the dataframe to the dictionary
    dfs[accession] = df

<ipython-input-7-87736d8d5c62>:17: DtypeWarning: Columns (29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + df_name + '.csv')


In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    preds = logits.argmax(axis=1)
    kappa = cohen_kappa_score(labels, preds, weights='quadratic')
    return {"cohen_kappa": kappa}

In [9]:
def preprocess(text):
    text=text.lower()
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
    #Replace &amp, &lt, &gt with &,<,> respectively
    text=text.replace(r'&amp;?',r'and')
    text=text.replace(r'&lt;',r'<')
    text=text.replace(r'&gt;',r'>')
    #remove hashtag sign
    text=re.sub(r"#","",text)   
    #remove mentions
    text = re.sub(r"(?:\@)\w+", '', text)
    text=re.sub(r"@","",text)
    #remove non ascii chars
    text=text.encode("ascii",errors="ignore").decode()
    #remove some puncts (except . ! ?)
    text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
    text=re.sub(r'[!]+','!',text)
    text=re.sub(r'[?]+','?',text)
    text=re.sub(r'[.]+','.',text)
    text=re.sub(r"'","",text)
    text=re.sub(r"\(","",text)
    text=re.sub(r"\)","",text)
    
    text=" ".join(text.split())
    return text

In [10]:

def detect_minority_majority_classes(df, label_column):

    class_counts = df[label_column].value_counts()
    minority_classes = class_counts[class_counts < class_counts.max()].index.tolist()
    majority_class = class_counts.idxmax()
    return minority_classes, majority_class

def augment_minority_class_text(df, text_column, label_column):
    augmented_texts = []
    aug = naw.RandomWordAug(action="swap")
    minority_classes, majority_class = detect_minority_majority_classes(df, label_column)
    print(df[label_column].value_counts())
    
    for minority_class in minority_classes:
        # Filter the dataframe to get only the minority class rows
        minority_df = df[df[label_column] == minority_class]
        majority_df = df[df[label_column] == majority_class]
        minority_count = len(minority_df)
        majority_count = len(majority_df)
        
        # Check if augmentation is required based on class imbalance
        if minority_count >= 0.4* majority_count:
            continue

        # Calculate the number of augmentations required
        num_augmentations = int(0.4 * majority_count) - minority_count
        
        # Augment the text of the minority class
        while num_augmentations > 0:
            for text in minority_df[text_column]:
                augmented_text = aug.augment(text)
                if augmented_text:
                    
                    augmented_texts.append((augmented_text[0], minority_class))  # Append augmented text with the minority class label
                    num_augmentations -= 1
                    if num_augmentations == 0:
                        break

    # Create a new dataframe with augmented texts
    augmented_df = pd.DataFrame(augmented_texts, columns=[text_column, label_column])
    
    # Concatenate the augmented dataframe with the original dataframe
    augmented_df = pd.concat([df, augmented_df], ignore_index=True)
    print(augmented_df[label_column].value_counts())
    return augmented_df



In [11]:
df_test = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/df_test.csv')

<ipython-input-11-35d9ebd626ae>:1: DtypeWarning: Columns (20,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/df_test.csv')


In [12]:
dfs_test = {}

In [13]:
# Unique values in the 'accession' column
unique_accessions = df_test['accession'].unique().tolist()
# Loop through the unique accessions
for accession in unique_accessions:
    # Filter the dataframe for the current accession
    grouped_df = df_test[df_test['accession'] == accession]
    # Create the dataframe name
    path = '/content/drive/MyDrive/NAEP_Comp/Final Results'
    df_name = 'df_test' + accession
    dfs_test[accession] = grouped_df

    # Save the dataframe as a CSV file
    grouped_df.to_csv(path + df_name + '.csv', index=False)

## Prompt 1 :VH134067

In [70]:
MODEL = "google/electra-base-discriminator"

In [81]:
df_1 = dfs['VH134067']
df_1_test = dfs_test['VH134067']

In [82]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]].fillna("")
df_1['labels'] = df_1['score_to_predict'] - 1
#df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
#df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

,student_id,predict_from,score_to_predict,labels
0,00DW6CD42b,the rule is the output numbers aretimed by 6,1,0
1,00Z4EpOsGR,345618243036,1,0
2,00kWAaE01z,wjnek nxkwndkqisnehebcbvbvbbbvbbbvfncn elsa,1,0
3,00tU2uw7WI,3456 18243036,1,0
4,00yF6z6nvp,they should be multiplyid,1,0


In [83]:
df_train, df_valid = train_test_split(df_1, test_size = 0.2, stratify = df_1['labels'], random_state=11 )

In [77]:
df_valid.shape

(8069, 4)

In [84]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, add_special_tokens = True, max_length = 100)   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])    

In [85]:
dataset_train = d1.from_pandas(df_train, preserve_index=False)
dataset_valid = d1.from_pandas(df_valid, preserve_index=False)
model = ElectraForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
model.cuda()
tokenizer = ElectraTokenizerFast.from_pretrained(MODEL, do_lower_case=True)
train_dataset = TrainerDataset(dataset_train["predict_from"],
                              dataset_train["labels"], tokenizer)
eval_dataset = TrainerDataset(dataset_valid["predict_from"],
                            dataset_valid["labels"], tokenizer) 

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

In [86]:
training_args = TrainingArguments(
  output_dir="./models/model_electra2_prompt_1",
  num_train_epochs=4,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  save_total_limit = 2,
  save_strategy = 'no',
  load_best_model_at_end=False ) # Make sure all batches are of equal size)
# Instantiate the Trainer class
trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Cohen Kappa
1,0.125900,0.106932,0.918799
2,0.096000,0.093998,0.932467
3,0.072100,0.090731,0.933663
4,0.051900,0.093695,0.936349


TrainOutput(global_step=4036, training_loss=0.09856225338988782, metrics={'train_runtime': 613.2332, 'train_samples_per_second': 210.517, 'train_steps_per_second': 6.582, 'total_flos': 6634098594288000.0, 'train_loss': 0.09856225338988782, 'epoch': 4.0})

In [87]:
pred, actual, _ = trainer.predict(eval_dataset)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.9363486503947526

In [88]:
test_indexes = dataset_valid['student_id']
score = dataset_valid['score_to_predict']

In [89]:
result_VH134067_electra = pd.DataFrame()
result_VH134067_electra['student_id'] = test_indexes
result_VH134067_electra['score'] = score
result_VH134067_electra['true_labels'] = actual +1 
result_VH134067_electra['predicted'] = pred_labels + 1

In [90]:
dfs['VH134067']

,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
0,00DW6CD42b,VH134067,1,the rule is the output numbers aretimed by 6,2017,1.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00Z4EpOsGR,VH134067,1,345618243036,2017,1.0,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00kWAaE01z,VH134067,1,wjnek nxkwndkqisnehebcbvbvbbbvbbbvfncn elsa,2017,1.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00tU2uw7WI,VH134067,1,3456 18243036,2017,2.0,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00yF6z6nvp,VH134067,1,they should be multiplyid,2017,3.0,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40338,zyKsRbfo13,VH134067,1,In this relationship you are counting by two's.,2019,1.0,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40339,zyTNyHw1KR,VH134067,1,In the input number they are counting by ones ...,2019,3.0,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40340,zyxL21hHpU,VH134067,1,they are all multiplesof each other,2019,3.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40341,zzDUDrGmNL,VH134067,1,each one is difrint,2019,1.0,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
df_copy = dfs['VH134067'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [92]:
merged_df = result_VH134067_electra.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,WvxU7AdnId,1,1,2,1,3.0,1.0,2.0,2.0,1.0
1,IO7pk7RDb2,1,1,1,1,5.0,1.0,2.0,1.0,2.0
2,hHQJerjX2r,2,2,2,2,4.0,1.0,2.0,2.0,2.0
3,LFpBpI5GeJ,1,1,1,1,7.0,1.0,2.0,2.0,2.0
4,VymBPtzmg6,1,1,1,1,5.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
8064,2DMXdMBZ7V,2,2,2,2,1.0,1.0,2.0,2.0,2.0
8065,DcwVuMHmPP,2,2,2,2,2.0,2.0,2.0,2.0,2.0
8066,ZPZHekSgwk,1,1,1,1,1.0,2.0,2.0,2.0,2.0
8067,sksTcBEUeW,1,1,1,1,7.0,1.0,2.0,2.0,2.0


In [93]:
merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH134067_electra.csv', index = False)

###Test Prediction

In [94]:
class TestDataset(Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, add_special_tokens = True, max_length = 100)   


    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],)   

In [95]:
#df_1_test['predict_from'] = df_1_test['predict_from'].apply(contractions.fix)
dataset_test = d1.from_pandas(df_1_test, preserve_index=False)
test_dataset = TestDataset(dataset_test["predict_from"], tokenizer) 



In [96]:
pred ,_,_ = trainer.predict(test_dataset)

In [97]:
pred_labels = np.argmax(pred, axis=1)

In [98]:
pred_labels

array([1, 0, 0, ..., 1, 0, 0])

In [99]:
df_1_test['score_to_predict'] = pred_labels + 1

<ipython-input-99-ae7249f2cc7a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_test['score_to_predict'] = pred_labels + 1


In [100]:
df_1_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH134067.csv', index = False)

## Prompt 2 : VH139380

In [720]:
MODEL = "distilbert-base-uncased"

In [721]:
df_1  = dfs['VH139380']
df_test = dfs_test['VH139380']

In [722]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]].fillna("")
df_1['labels'] = df_1['score_to_predict'] - 1
#df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

,student_id,predict_from,score_to_predict,labels
0,00H97obW4T,the pattern continues by adding the same numbe...,2,1
1,00rGiuq5ny,it is adding by threes,3,2
2,01gXWb2lDU,2 4 6 8 10 12 14 16 17,2,1
3,01p9XCjUlY,3,2,1
4,01tNvcide9,5+ 3 8+3 +3+#+#+#++#+#+#+#++##3333333333333333...,2,1


In [723]:
train_df, test_df = train_test_split(df_1, test_size=0.2, random_state=11)
train_dataset = d1.from_pandas(train_df, preserve_index=True)
test_dataset = d1.from_pandas(test_df, preserve_index=True)

In [724]:
test_df.shape

(3632, 4)

In [725]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_cases = True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [726]:
model.resize_token_embeddings(len(tokenizer))

def tokenize_function(examples):
    return tokenizer(examples["predict_from"], padding="max_length", truncation=True,
                     add_special_tokens=True, max_length = 180)



In [727]:
train_datasets = train_dataset.map(tokenize_function, batched=True)
test_datasets = test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/14525 [00:00<?, ? examples/s]

Map:   0%|          | 0/3632 [00:00<?, ? examples/s]

In [115]:
train_datasets, test_datasets

(Dataset({
     features: ['student_id', 'predict_from', 'score_to_predict', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
     num_rows: 14525
 }),
 Dataset({
     features: ['student_id', 'predict_from', 'score_to_predict', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
     num_rows: 3632
 }))

In [728]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  num_train_epochs= 4,
                                  save_total_limit = 2,
                                  save_strategy = 'epoch',
                                  load_best_model_at_end=False
                                  )


trainer = Trainer(model=model,args=training_args,train_dataset=train_datasets,  eval_dataset=test_datasets,
                  compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Cohen Kappa
1,0.533100,0.485599,0.795913
2,0.443400,0.478156,0.797311
3,0.388600,0.481511,0.805166
4,0.335900,0.504607,0.801806


TrainOutput(global_step=1816, training_loss=0.4252482846970075, metrics={'train_runtime': 251.8641, 'train_samples_per_second': 230.68, 'train_steps_per_second': 7.21, 'total_flos': 2705798360916000.0, 'train_loss': 0.4252482846970075, 'epoch': 4.0})

In [729]:
pred, actual, _ = trainer.predict(test_datasets)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.8018060883847691

In [730]:
test_indexes = test_dataset['student_id']
score = test_dataset['score_to_predict']

In [731]:
result_VH139380_distill = pd.DataFrame()
result_VH139380_distill['student_id'] = test_indexes
result_VH139380_distill['score'] = score
result_VH139380_distill['true_labels'] = actual +1 
result_VH139380_distill['predicted'] = pred_labels + 1

In [735]:
df_copy = dfs['VH139380'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [736]:
merged_df = result_VH139380_distill.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,4MzPwZaG7f,2,2,2,2,1.0,2.0,2.0,2.0,2.0
1,tcPCrIiP6G,2,2,2,2,1.0,1.0,2.0,2.0,2.0
2,xxg6MIK2O8,3,3,3,3,1.0,2.0,1.0,1.0,2.0
3,3rpj346MgG,2,2,2,2,1.0,1.0,2.0,2.0,2.0
4,T0rU9bteJ6,3,3,3,3,3.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...
3627,nc8uD4MX47,3,3,3,3,1.0,2.0,2.0,2.0,2.0
3628,qfTPAuc9uy,3,3,3,3,1.0,1.0,2.0,2.0,2.0
3629,T6SQ2bbY9b,1,1,2,1,7.0,1.0,1.0,1.0,2.0
3630,oItYLwmUul,3,3,3,3,7.0,1.0,2.0,2.0,2.0


In [737]:

merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH139380_distill.csv', index = False)
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,4MzPwZaG7f,2,2,2,2,1.0,2.0,2.0,2.0,2.0
1,tcPCrIiP6G,2,2,2,2,1.0,1.0,2.0,2.0,2.0
2,xxg6MIK2O8,3,3,3,3,1.0,2.0,1.0,1.0,2.0
3,3rpj346MgG,2,2,2,2,1.0,1.0,2.0,2.0,2.0
4,T0rU9bteJ6,3,3,3,3,3.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...
3627,nc8uD4MX47,3,3,3,3,1.0,2.0,2.0,2.0,2.0
3628,qfTPAuc9uy,3,3,3,3,1.0,1.0,2.0,2.0,2.0
3629,T6SQ2bbY9b,1,1,2,1,7.0,1.0,1.0,1.0,2.0
3630,oItYLwmUul,3,3,3,3,7.0,1.0,2.0,2.0,2.0


In [738]:
true_labels = merged_df['true_labels']
pred = merged_df['predicted']

In [739]:
cohen_kappa_score(pred, true_labels, weights = 'quadratic')

0.8018060883847691

### Test Predictions

In [740]:
test_dataset = d1.from_pandas(df_test, preserve_index=False)

In [741]:
test_datasets = test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/2018 [00:00<?, ? examples/s]

In [150]:
test_datasets

Dataset({
    features: ['student_id', 'accession', 'score_to_predict', 'predict_from', 'year', 'srace10', 'dsex', 'accom2', 'iep', 'lep', 'rater_1', 'pta_rtr1', 'ptb_rtr1', 'ptc_rtr1', 'composite', 'score', 'assigned_score', 'ee_use', 'parsed_xml_v1', 'parsed_xml_v2', 'parsed_xml_v3', 'source1', 'source2', 'source3', 'source4', 'target1', 'target2', 'target3', 'target4', 'eliminations', 'selected', 'eliminated', 'selected1', 'selected2', 'selected3', 'selected4', 'eliminated1', 'eliminated2', 'eliminated3', 'eliminated4', 'selected1.1', 'selected2.1', 'eliminated1.1', 'eliminated2.1', 'partA_response_val', 'partB_response_val', 'partB_eliminations', 'input_ids', 'attention_mask'],
    num_rows: 2018
})

In [742]:
pred, _, _ = trainer.predict(test_datasets)
pred_labels = np.argmax(pred, axis=1)
pred_labels

array([2, 1, 2, ..., 0, 2, 1])

In [743]:
df_test_score = df_test.copy()
df_test_score['score_to_predict'] = pred_labels + 1
df_test_score['score_to_predict'].value_counts()

3    902
2    843
1    273
Name: score_to_predict, dtype: int64

In [744]:
df_test_score.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH139380.csv', index = False)

## Prompt 3 : VH266015

In [232]:
df_1 = dfs['VH266015']
df_test = dfs['VH266015']

In [233]:
df_1 = df_1.dropna(subset=['predict_from'])

In [234]:
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]]
#df_1['predict_from'] = df_1["predict_from"].apply(preprocess)
#df_1['predict_from'] = df_1["predict_from"].apply(contractions.fix)
df_1

,student_id,predict_from,score_to_predict
0,003GxiQOSi,It has to be greater then 7/6 to be equivilent.,1
1,005PFKpBb1,a and b must both be integers,1
2,00I2EDxQjW,They equal the same.,2
3,00O6qASsRq,they have to be equal and the division or mult...,2
4,00qAKes5d7,A and B have to be order pairs because they're...,1
...,...,...,...
15223,zxzi41SWvk,12,1
15224,zyEwTJuoyT,they both have the numbers switched around.,1
15225,zyeYRoq6ob,58,1
15226,zzSTXhdXX0,56,1


In [235]:
df_train, df_test= train_test_split(df_1, test_size=0.2, random_state=42, stratify = df_1['score_to_predict'])

In [236]:
df_test.shape

(3046, 3)

In [221]:

meth_bert_tokenizer = AutoTokenizer.from_pretrained('tbs17/MathBERT', do_lower_case=True)

In [223]:
import time
import datetime
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm



def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def get_optimizer_and_scheduler(model, total_steps, lr=2e-5, weight_decay=0.01):
    # Apply weight decay to all parameters beside the biases or LayerNorm weights
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': weight_decay},
        {
            'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
        }
    ]
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        # Warmup learning rate for first 10% of training steps
        num_warmup_steps=int(0.10 * total_steps), 
        num_training_steps=total_steps,
    )
    return optimizer, scheduler

In [244]:
from transformers import BertModel
import torch.nn as nn

class BertCNNLSTMClassifier(BertModel):
    def __init__(self, config):
        super().__init__(config)
        
        self.lstm_hidden_size = 256
        self.lstm_num_layers = 4
        self.cnn_num_filters = 256
        self.cnn_kernel_size = 3
        
        # Add a linear classifier that maps the concatenated representation to the unnormalized
        # output probabilities for each class (logits).
        self.classifier = nn.Linear(config.hidden_size + self.lstm_hidden_size * 2 + self.cnn_num_filters, config.num_labels)
        
        # Bidirectional LSTM layer
        self.lstm = nn.LSTM(config.hidden_size, self.lstm_hidden_size, num_layers=self.lstm_num_layers, batch_first=True, bidirectional=True)
        
        # CNN layer
        self.cnn = nn.Conv1d(
            in_channels=self.lstm_hidden_size * 2,
            out_channels=self.cnn_num_filters,
            kernel_size=self.cnn_kernel_size,
            padding=(self.cnn_kernel_size - 1) // 2
        )
        
        # Activation function
        self.activation = nn.ReLU()
        
        # Loss function
        self.loss = torch.nn.CrossEntropyLoss()

    def forward(self, labels=None, **kwargs):
        outputs = super().forward(**kwargs)
        
        # Pass BERT's [CLS] token representation to the LSTM layer
        cls_token_repr = outputs.pooler_output
        lstm_output, _ = self.lstm(cls_token_repr.unsqueeze(1))
        
        # Reshape the LSTM output for CNN
        lstm_output = lstm_output.permute(0, 2, 1)  # Reshape for CNN
        
        # Pass the LSTM output through the CNN layer
        cnn_output = self.cnn(lstm_output)
        cnn_output = cnn_output.squeeze(2)  # Remove the last dimension
        
        # Reshape the LSTM output for concatenation
        lstm_output = lstm_output.squeeze(2)
        
        # Concatenate the [CLS] token representation, LSTM output, and CNN output
        concatenated_repr = torch.cat((cls_token_repr, lstm_output, cnn_output), dim=1)
        
        # Pass the concatenated representation to the linear classifier
        logits = self.classifier(concatenated_repr)
        
        if labels is not None:
            outputs = (logits, self.loss(logits, labels))
        else:
            outputs = (logits,)
        
        return outputs


In [162]:
df_balanced = augment_minority_class_text(df_train, 'predict_from','score_to_predict')

1    9990
2    2180
3      12
Name: score_to_predict, dtype: int64
1    9990
2    6993
3    6993
Name: score_to_predict, dtype: int64


In [245]:
train_inputs = df_train.predict_from.values.tolist()
train_labels = df_train.score_to_predict.values - 1
train_indexes = df_test.student_id.values.tolist()
valid_inputs = df_test.predict_from.values.tolist()
valid_labels = df_test.score_to_predict.values - 1
tokenized_inputs_train = meth_bert_tokenizer(train_inputs, add_special_tokens=True,
        padding='max_length',
        max_length=100,
        return_tensors='pt',
        truncation=True)
tokenized_inputs_valid = meth_bert_tokenizer(valid_inputs, add_special_tokens=True,
        padding='max_length',
        max_length=100,
        return_tensors='pt',
        truncation=True)
train_input_ids = tokenized_inputs_train['input_ids']
train_attention_masks = tokenized_inputs_train['attention_mask']
valid_input_ids = tokenized_inputs_valid['input_ids']
valid_attention_masks = tokenized_inputs_valid['attention_mask']



In [246]:
train_input_ids.shape, train_attention_masks.shape, valid_input_ids.shape, valid_attention_masks.shape, train_labels.shape, valid_labels.shape

(torch.Size([12182, 100]),
 torch.Size([12182, 100]),
 torch.Size([3046, 100]),
 torch.Size([3046, 100]),
 (12182,),
 (3046,))

In [247]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [248]:
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(valid_labels)  # Add a dimension to match the other tensors

# Create the DataLoader for our training set.
train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=32)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(valid_input_ids, valid_attention_masks, validation_labels)
validation_dataloader = DataLoader(validation_data, shuffle=False, batch_size=32)


In [249]:
mathbert = BertCNNLSTMClassifier.from_pretrained("tbs17/MathBERT", num_labels=df['assigned_score'].nunique()) 

Some weights of the model checkpoint at tbs17/MathBERT were not used when initializing BertCNNLSTMClassifier: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertCNNLSTMClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertCNNLSTMClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [250]:
def train_model(model, epochs, train_dataloader, validation_dataloader):
    # Use GPU, if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    kappa_score_max = 0
    best_model = model

    # Setup optimizer and LR scheduler 
    total_steps = len(train_dataloader) * epochs
    optimizer, scheduler = get_optimizer_and_scheduler(
        model, total_steps, lr=5e-5, weight_decay=0.01)

    loss_values = []
    eval_accs = []
        # Tracking variables
    

    for epoch in range(0, epochs):
        t0 = time.time()

        total_loss = 0
        model.train()
        predictions, true_labels = [], []

        with tqdm(train_dataloader, unit="batch") as train_pbar:
            for batch in train_pbar:
                train_pbar.set_description(f"Training (epoch {epoch + 1})")
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                model.zero_grad()        

              
                outputs = model(
                    input_ids=b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels )
                

                _, loss = outputs


                total_loss += loss.item()


                loss.backward()


                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                optimizer.step()

                # Update the learning rate.
                scheduler.step()

        avg_train_loss = total_loss / len(train_dataloader)            
        
        loss_values.append(avg_train_loss)

        print("  * Average training loss: {0:.2f}".format(avg_train_loss))
        print("  * Training epoch took: {:}".format(format_time(time.time() - t0)))
            
        print("Running Validation...")

        t0 = time.time()
        model.eval()

        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            
            with torch.no_grad():        

                outputs = model(
                    input_ids=b_input_ids, 
                    attention_mask=b_input_mask)
            

            logits = outputs[0]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            # Calculate the accuracy for this batch of test sentences.
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy
            # Track the number of batches
            nb_eval_steps += 1
            predictions.append(logits)
            true_labels.append(label_ids)
       
        predictions = np.concatenate(predictions)
        true_labels = np.concatenate(true_labels)

        predicted_labels = np.argmax(predictions, axis=1)
        kappa_score = cohen_kappa_score(true_labels, predicted_labels, labels=None, weights= 'quadratic', sample_weight=None)
      
        
        
        if kappa_score > kappa_score_max :
          kappa_score_max = kappa_score
          predicted_labels_max = predicted_labels
          best_model = model
        avg_eval_acc = eval_accuracy/nb_eval_steps
        print("  * Accuracy: {0:.2f}".format(avg_eval_acc))
        print("  * Validation took: {:}".format(format_time(time.time() - t0)))
        print("Kappa Score = " + str(kappa_score))
        eval_accs.append(avg_eval_acc)
    print("Training complete!")
    return best_model,kappa_score_max, predicted_labels_max, true_labels

In [251]:
best_model, score, predicted_labels, true_labels = train_model(model=mathbert,epochs=5,train_dataloader=train_dataloader,
  validation_dataloader=validation_dataloader)
print(score)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training (epoch 1): 100%|██████████| 381/381 [00:55<00:00,  6.88batch/s]


  * Average training loss: 0.25
  * Training epoch took: 0:00:55
Running Validation...
  * Accuracy: 0.95
  * Validation took: 0:00:04
Kappa Score = 0.822518703335039


Training (epoch 2): 100%|██████████| 381/381 [00:55<00:00,  6.88batch/s]


  * Average training loss: 0.13
  * Training epoch took: 0:00:55
Running Validation...
  * Accuracy: 0.95
  * Validation took: 0:00:04
Kappa Score = 0.8287238141120288


Training (epoch 3): 100%|██████████| 381/381 [00:55<00:00,  6.87batch/s]


  * Average training loss: 0.08
  * Training epoch took: 0:00:55
Running Validation...
  * Accuracy: 0.96
  * Validation took: 0:00:04
Kappa Score = 0.8521520975462038


Training (epoch 4): 100%|██████████| 381/381 [00:55<00:00,  6.88batch/s]


  * Average training loss: 0.05
  * Training epoch took: 0:00:55
Running Validation...
  * Accuracy: 0.96
  * Validation took: 0:00:04
Kappa Score = 0.852317849616801


Training (epoch 5): 100%|██████████| 381/381 [00:55<00:00,  6.88batch/s]


  * Average training loss: 0.03
  * Training epoch took: 0:00:55
Running Validation...
  * Accuracy: 0.96
  * Validation took: 0:00:04
Kappa Score = 0.8610572153688228
Training complete!
0.8610572153688228


In [252]:
cohen_kappa_score(predicted_labels, true_labels, weights = 'quadratic')

0.8610572153688228

In [253]:
test_indexes = df_test['student_id']
score = df_test['score_to_predict']

In [254]:
result_VH266510_mathbert = pd.DataFrame()
result_VH266510_mathbert['student_id'] = test_indexes
result_VH266510_mathbert['score'] = score
result_VH266510_mathbert['true_labels'] = true_labels +1 
result_VH266510_mathbert['predicted'] = predicted_labels + 1

In [255]:
df_copy = dfs['VH266015'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [256]:
df_copy

,student_id,score_to_predict,srace10,dsex,accom2,iep,lep
0,003GxiQOSi,1,7.0,1.0,2.0,2.0,2.0
1,005PFKpBb1,1,1.0,2.0,2.0,2.0,2.0
2,00I2EDxQjW,2,1.0,1.0,2.0,2.0,2.0
3,00O6qASsRq,2,3.0,2.0,2.0,2.0,2.0
4,00qAKes5d7,1,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...
15223,zxzi41SWvk,1,3.0,2.0,1.0,2.0,1.0
15224,zyEwTJuoyT,1,2.0,2.0,2.0,2.0,2.0
15225,zyeYRoq6ob,1,3.0,2.0,2.0,2.0,2.0
15226,zzSTXhdXX0,1,3.0,1.0,1.0,1.0,2.0


In [257]:
merged_df = result_VH266510_mathbert.merge(df_copy, on="student_id", how="inner")

merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH266015_mathbert.csv', index = False)
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,6Y3sEgygjg,1,1,1,1,1.0,2.0,2.0,2.0,2.0
1,QAkbQSrfsK,1,1,1,1,1.0,1.0,1.0,1.0,2.0
2,3MoC2DvEIv,2,2,2,2,2.0,1.0,2.0,1.0,2.0
3,AtE1JOXpZJ,2,2,2,2,1.0,1.0,2.0,2.0,2.0
4,cPDpwwdvxX,1,1,1,1,4.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
3041,sk59edHGAk,1,1,1,1,2.0,1.0,2.0,2.0,2.0
3042,LRZIUaiBEo,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3043,yRWgHTRXTA,1,1,1,1,3.0,2.0,2.0,2.0,2.0
3044,1ReWFfiRVx,1,1,1,1,1.0,2.0,2.0,2.0,2.0


### Test Predictions



In [258]:
df_test = dfs_test['VH266015']
df_test["predict_from"]= df_test["predict_from"].fillna('N/A')
#df_1['predict_from'] = df_1["predict_from"].apply(contractions.fix)
df_test


,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
6501,03yr8ZiXhl,VH266015,NaN,43,2019,4,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6502,050C4NmLo4,VH266015,NaN,they have to be equal to each other,2019,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6503,0A9Z4sthfJ,VH266015,NaN,76,2019,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6504,0Duy5W7y7W,VH266015,NaN,its true,2019,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6505,0FhScpJyU5,VH266015,NaN,that a & b = 7/6,2019,2,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8188,zt2lXjziqJ,VH266015,NaN,the same values,2019,7,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8189,zuZm8Z2XW4,VH266015,NaN,The values of a and b should be the same becau...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8190,zvkEeSn5ur,VH266015,NaN,One thing that true about values a and b is th...,2019,4,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8191,zx0La543X4,VH266015,NaN,The values of a and b must be equal to 7/6 and...,2019,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_test['predict_from'].isna().sum()

0

In [259]:
test_inputs = df_test.predict_from.values.tolist()

In [260]:
tokenized_inputs_test = meth_bert_tokenizer(test_inputs, add_special_tokens=True,
        padding='max_length',
        max_length=100,
        return_tensors='pt',
        truncation=True)

In [261]:
test_input_ids = tokenized_inputs_test['input_ids']
test_attention_masks = tokenized_inputs_test['attention_mask']

In [263]:
test_input_ids.shape

torch.Size([1692, 100])

In [262]:
test_data = TensorDataset(test_input_ids, test_attention_masks)
test_loader = DataLoader(test_data, shuffle=True, batch_size=32)

In [264]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [265]:
t0 = time.time()
predictions = []
best_model.eval()
for batch in test_loader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch  # Remove the unused b_labels
    
    with torch.no_grad():
        outputs = best_model(input_ids=b_input_ids, attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)

predictions = np.concatenate(predictions)
predicted_labels = np.argmax(predictions, axis=1) + 1
print(np.unique(predicted_labels))

[1 2 3]


In [266]:
len(predicted_labels)

1692

In [267]:
print(np.unique(predicted_labels, return_counts=True))

(array([1, 2, 3]), array([1367,  323,    2]))


In [268]:
df_test['score_to_predict'] = predicted_labels
df_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH266015.csv', index = False)
df_test

,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
6501,03yr8ZiXhl,VH266015,2,43,2019,4,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6502,050C4NmLo4,VH266015,1,they have to be equal to each other,2019,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6503,0A9Z4sthfJ,VH266015,1,76,2019,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6504,0Duy5W7y7W,VH266015,1,its true,2019,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6505,0FhScpJyU5,VH266015,1,that a & b = 7/6,2019,2,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8188,zt2lXjziqJ,VH266015,1,the same values,2019,7,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8189,zuZm8Z2XW4,VH266015,1,The values of a and b should be the same becau...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8190,zvkEeSn5ur,VH266015,1,One thing that true about values a and b is th...,2019,4,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8191,zx0La543X4,VH266015,1,The values of a and b must be equal to 7/6 and...,2019,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Prompt 4 : VH266510

In [745]:
MODEL = "google/electra-base-discriminator"

In [746]:
df_1 = dfs['VH266510']
df_1_test = dfs_test['VH266510']

In [747]:
df_1 = df_1.dropna(subset = ['predict_from'])

In [748]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]]
df_1['labels'] = df_1['score_to_predict'] - 1
df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

<ipython-input-748-72af636fc434>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['labels'] = df_1['score_to_predict'] - 1


,student_id,predict_from,score_to_predict,labels
0,00Mne1VoXJ,well i know this because if you have the slope...,1,0
1,00Nk4ToFv7,the case in which the slopes of the lines are ...,3,2
2,00YOvNfuNq,when the two slopes are parallel the sides mus...,1,0
3,00iP2sLaGr,i know this is the right answer because if the...,3,2
4,010hgozYgu,the have to have the same points in order to i...,1,0


In [754]:
df_train, df_valid = train_test_split(df_1, test_size = 0.2, stratify = df_1['labels'], random_state=11 )

In [750]:
df_valid.shape

(7734, 4)

In [755]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, max_length = 248, add_special_tokens = True)   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])    

In [756]:
dataset_train = d1.from_pandas(df_train, preserve_index=False)
dataset_valid = d1.from_pandas(df_valid, preserve_index=False)
model = ElectraForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
model.cuda()
tokenizer = ElectraTokenizerFast.from_pretrained(MODEL, do_lower_case=True)
train_dataset = TrainerDataset(dataset_train["predict_from"],
                              dataset_train["labels"], tokenizer)
eval_dataset = TrainerDataset(dataset_valid["predict_from"],
                            dataset_valid["labels"], tokenizer) 

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

In [757]:
training_args = TrainingArguments(
  output_dir="./models/model_electra2_prompt_4",
  num_train_epochs=4,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  save_total_limit = 2,
  save_strategy = 'epoch',
  load_best_model_at_end=False ) # Make sure all batches are of equal size)
# Instantiate the Trainer class
trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Cohen Kappa
1,0.307800,0.238179,0.815339
2,0.205000,0.206921,0.851166
3,0.159000,0.213922,0.850038
4,0.116300,0.238070,0.856477


TrainOutput(global_step=3868, training_loss=0.18619870301239738, metrics={'train_runtime': 1033.8889, 'train_samples_per_second': 119.676, 'train_steps_per_second': 3.741, 'total_flos': 1.1890405738738296e+16, 'train_loss': 0.18619870301239738, 'epoch': 4.0})

In [758]:
pred, actual, _ = trainer.predict(eval_dataset)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.8564767336592478

In [759]:
test_indexes = dataset_valid['student_id']
score = dataset_valid['score_to_predict']

In [760]:
result_VH266510_electra = pd.DataFrame()
result_VH266510_electra['student_id'] = test_indexes
result_VH266510_electra['score'] = score
result_VH266510_electra['true_labels'] = actual +1 
result_VH266510_electra['predicted'] = pred_labels + 1

In [761]:
df_copy = dfs['VH266510'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [762]:
merged_df = result_VH266510_electra.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,40G7HpdWil,1,1,1,1,1.0,2.0,2.0,2.0,2.0
1,t5mZxHhMlZ,2,2,1,2,4.0,1.0,2.0,2.0,2.0
2,CZKDaUeL4G,1,1,1,1,6.0,2.0,2.0,2.0,2.0
3,USvVyxPJ1o,1,1,1,1,1.0,2.0,2.0,2.0,2.0
4,a8RR08VFjq,1,1,1,1,3.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
7729,G5a8hR0sy0,1,1,1,1,1.0,1.0,2.0,2.0,2.0
7730,HC8bCFhSkS,3,3,3,3,1.0,1.0,2.0,2.0,2.0
7731,9IOoYnCO3Y,1,1,1,1,1.0,2.0,2.0,2.0,2.0
7732,f6yTPqc2X5,3,3,3,3,4.0,1.0,2.0,2.0,2.0


In [763]:
merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH266510_electra.csv', index = False)

###Test Prediction

In [764]:
class TestDataset(Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, max_length = 248, add_special_tokens = True)   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],)   

In [765]:
df_1_test['predict_from'] = df_1_test['predict_from'].fillna(df_1_test['parsed_xml_v1'])
df_1_test['predict_from'] = df_1_test['predict_from'].fillna('N/A')
df_1_test['predict_from'].isna().sum()

0

In [766]:
df_1_test['predict_from'] = df_1_test['predict_from'].apply(preprocess)
df_1_test['predict_from'] = df_1_test['predict_from'].apply(contractions.fix)
dataset_test = d1.from_pandas(df_1_test, preserve_index=False)
test_dataset = TestDataset(dataset_test["predict_from"], tokenizer) 



In [767]:
pred ,_,_ = trainer.predict(test_dataset)

In [768]:
pred_labels = np.argmax(pred, axis=1)

In [769]:
pred_labels

array([2, 0, 0, ..., 0, 2, 0])

In [770]:
df_1_test['score_to_predict'] = pred_labels + 1

In [ ]:
df_1_test[['score_to_predict', 'partB_response_val']]

,score_to_predict,partB_response_val
8193,3,NaN
8194,1,NaN
8195,1,NaN
8196,3,NaN
8197,1,NaN
...,...,...
12484,1,NaN
12485,1,NaN
12486,1,NaN
12487,3,NaN


In [771]:
df_1_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH266510.csv', index = False)

## Prompt 5 : VH269384

In [341]:
MODEL = "google/electra-base-discriminator"

In [351]:
df_1 = dfs['VH269384']
df_1_test = dfs_test['VH269384']

In [352]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]]
df_1['labels'] = df_1['score_to_predict'] - 1
#df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
#df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

<ipython-input-352-14cb78c66525>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['labels'] = df_1['score_to_predict'] - 1


,student_id,predict_from,score_to_predict,labels
0,001bD4Xz7O,He might get unlucky,1,0
1,00V5qkua6L,"9 is greater than three, so is 5. If you take ...",3,2
2,00f5wfNzcb,mm,1,0
3,00gs0l13rA,becase it help you a lote,1,0
4,00i1wiGLVd,no because there is the same amount of cards a...,1,0


In [366]:
df_train, df_valid = train_test_split(df_1, test_size = 0.2, stratify = df_1['labels'], random_state=11 )

In [347]:
df_valid.shape


(3164, 4)

In [367]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, add_special_tokens = True, max_length = 100)   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])    

In [368]:
dataset_train = d1.from_pandas(df_train, preserve_index=False)
dataset_valid = d1.from_pandas(df_valid, preserve_index=False)
model = ElectraForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
model.cuda()
tokenizer = ElectraTokenizerFast.from_pretrained(MODEL, do_lower_case=True)
model.resize_token_embeddings(len(tokenizer))
train_dataset = TrainerDataset(dataset_train["predict_from"],
                              dataset_train["labels"], tokenizer)
eval_dataset = TrainerDataset(dataset_valid["predict_from"],
                            dataset_valid["labels"], tokenizer) 

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

In [369]:
training_args = TrainingArguments(
  output_dir="./models/model_electra2_prompt_4",
  num_train_epochs=10,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  save_total_limit = 2,
  save_strategy = 'epoch',
  load_best_model_at_end=False ) # Make sure all batches are of equal size)
# Instantiate the Trainer class
trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Cohen Kappa
1,No log,0.271029,0.807023
2,0.342900,0.298819,0.803065
3,0.240900,0.243137,0.843518
4,0.185200,0.278087,0.839614
5,0.185200,0.286633,0.849045
6,0.149400,0.345803,0.849333
7,0.109700,0.391448,0.857872
8,0.086800,0.403782,0.855669
9,0.064900,0.433780,0.860437
10,0.064900,0.455764,0.857389


TrainOutput(global_step=3960, training_loss=0.15481777480154327, metrics={'train_runtime': 624.0926, 'train_samples_per_second': 202.774, 'train_steps_per_second': 6.345, 'total_flos': 6503320901070000.0, 'train_loss': 0.15481777480154327, 'epoch': 10.0})

In [370]:
pred, actual, _ = trainer.predict(eval_dataset)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.8573894306090274

In [371]:
test_indexes = dataset_valid['student_id']
score = dataset_valid['score_to_predict']

In [372]:
result_VH269384_electra = pd.DataFrame()
result_VH269384_electra['student_id'] = test_indexes
result_VH269384_electra['score'] = score
result_VH269384_electra['true_labels'] = actual +1 
result_VH269384_electra['predicted'] = pred_labels + 1

In [373]:
df_copy = dfs['VH269384'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [374]:
merged_df = result_VH269384_electra.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,0wcVHaIuEr,1,1,1,1,3.0,2.0,2.0,2.0,2.0
1,rRFHFzsWvX,1,1,1,1,3.0,2.0,2.0,2.0,2.0
2,DSVdfino8I,1,1,1,1,7.0,2.0,2.0,2.0,2.0
3,LOZOMLgQcg,1,1,1,1,1.0,1.0,2.0,2.0,2.0
4,29N7onuob4,1,1,1,1,2.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
3159,WGcnINB0hJ,1,1,1,1,1.0,2.0,2.0,2.0,2.0
3160,adSuoV56l9,1,1,1,1,1.0,2.0,2.0,2.0,2.0
3161,h8O7cS1FfB,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3162,B3PLeSnuV1,1,1,1,1,1.0,2.0,2.0,2.0,2.0


In [375]:
merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH269384_electra.csv', index = False)

###Test Prediction

In [376]:
class TestDataset(Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, max_length = 100, add_special_tokens = True)
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],)   

In [377]:
df_1_test['predict_from'] = df_1_test['predict_from'].fillna(df_1_test['parsed_xml_v1'])
df_1_test['predict_from'] = df_1_test['predict_from'].fillna('N/A')
df_1_test['predict_from'].isna().sum()

0

In [379]:
df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
#df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
dataset_test = d1.from_pandas(df_1_test, preserve_index=False)
test_dataset = TestDataset(dataset_test["predict_from"], tokenizer) 



<ipython-input-379-a76d27727607>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['predict_from'] = df_1['predict_from'].apply(preprocess)


In [380]:
pred ,_,_ = trainer.predict(test_dataset)

In [381]:
pred_labels = np.argmax(pred, axis=1)

In [382]:
pred_labels

array([0, 0, 2, ..., 0, 2, 2])

In [383]:
df_1_test['score_to_predict'] = pred_labels + 1

In [384]:
df_1_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH269384.csv', index = False)

## Prompt 6 : VH271613

In [892]:
df_1 = dfs['VH271613']
df_test = dfs['VH271613']

In [894]:
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]]
#df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
#df_1['predict_from'] = df_1["predict_from"].apply(contractions.fix)
df_1

,student_id,predict_from,score_to_predict
0,00DX05DRh5,it is not true because phil's is 3 times yonge...,1
1,00N3yEYMAy,because is zach is 4 years older than tim that...,1
2,00TuatdAM2,because now zach will be older than tim and alex,1
3,00wXPqjAxM,"because we older ,not yonger",1
4,00wtajhH6E,because i,1
...,...,...,...
27843,zyUjt9xqkg,because phil's is 3 years older than alex's age,1
27844,zzBMPZ2ulP,because what if he was exp: he's like 16 and zacj,1
27845,zzEWXuRyL6,ijhgghhhhhhhhhhhjhu,1
27846,zzfwDRPzZC,it will not be true because zach will be older...,1


In [895]:
df_train, df_test= train_test_split(df_1, test_size=0.2, random_state=11, stratify = df_1['score_to_predict'])

In [896]:

meth_bert_tokenizer = AutoTokenizer.from_pretrained('tbs17/MathBERT', do_lower_case=True)
#model.resize_token_embeddings(len(meth_bert_tokenizer))

In [776]:
import time
import datetime
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm



def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def get_optimizer_and_scheduler(model, total_steps, lr=2e-5, weight_decay=0.01):
    # Apply weight decay to all parameters beside the biases or LayerNorm weights
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': weight_decay},
        {
            'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
        }
    ]
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        # Warmup learning rate for first 10% of training steps
        num_warmup_steps=int(0.10 * total_steps), 
        num_training_steps=total_steps,
    )
    return optimizer, scheduler

In [897]:
from transformers import BertModel
import torch.nn as nn

class BertCNNLSTMClassifier(BertModel):
    def __init__(self, config):
        super().__init__(config)
        
        self.lstm_hidden_size = 256
        self.lstm_num_layers = 4
        self.cnn_num_filters = 256
        self.cnn_kernel_size = 3
        
        # Add a linear classifier that maps the concatenated representation to the unnormalized
        # output probabilities for each class (logits).
        self.classifier = nn.Linear(config.hidden_size + self.lstm_hidden_size * 2 + self.cnn_num_filters, config.num_labels)
        
        # Bidirectional LSTM layer
        self.lstm = nn.LSTM(config.hidden_size, self.lstm_hidden_size, num_layers=self.lstm_num_layers, batch_first=True, bidirectional=True)
        
        # CNN layer
        self.cnn = nn.Conv1d(
            in_channels=self.lstm_hidden_size * 2,
            out_channels=self.cnn_num_filters,
            kernel_size=self.cnn_kernel_size,
            padding=(self.cnn_kernel_size - 1) // 2
        )
        
        # Activation function
        self.activation = nn.ReLU()
        
        # Loss function
        self.loss = torch.nn.CrossEntropyLoss()

    def forward(self, labels=None, **kwargs):
        outputs = super().forward(**kwargs)
        
        # Pass BERT's [CLS] token representation to the LSTM layer
        cls_token_repr = outputs.pooler_output
        lstm_output, _ = self.lstm(cls_token_repr.unsqueeze(1))
        
        # Reshape the LSTM output for CNN
        lstm_output = lstm_output.permute(0, 2, 1)  # Reshape for CNN
        
        # Pass the LSTM output through the CNN layer
        cnn_output = self.cnn(lstm_output)
        cnn_output = cnn_output.squeeze(2)  # Remove the last dimension
        
        # Reshape the LSTM output for concatenation
        lstm_output = lstm_output.squeeze(2)
        
        # Concatenate the [CLS] token representation, LSTM output, and CNN output
        concatenated_repr = torch.cat((cls_token_repr, lstm_output, cnn_output), dim=1)
        
        # Pass the concatenated representation to the linear classifier
        logits = self.classifier(concatenated_repr)
        
        if labels is not None:
            outputs = (logits, self.loss(logits, labels))
        else:
            outputs = (logits,)
        
        return outputs


In [863]:
df_balanced = augment_minority_class_text(df_train, 'predict_from','score_to_predict')

1    21368
3      687
2      223
Name: score_to_predict, dtype: int64
1    21368
3     8547
2     8547
Name: score_to_predict, dtype: int64


In [908]:
train_inputs = df_train.predict_from.values.tolist()
train_labels = df_train.score_to_predict.values - 1
train_indexes = df_test.student_id.values.tolist()
valid_inputs = df_test.predict_from.values.tolist()
valid_labels = df_test.score_to_predict.values - 1
tokenized_inputs_train = meth_bert_tokenizer(train_inputs, add_special_tokens=True,
        padding=True,
        return_tensors='pt',
        )
tokenized_inputs_valid = meth_bert_tokenizer(valid_inputs, add_special_tokens=True,
        padding=True, max_length = 178, truncation = True,
        return_tensors='pt',
        )
train_input_ids = tokenized_inputs_train['input_ids']
train_attention_masks = tokenized_inputs_train['attention_mask']
valid_input_ids = tokenized_inputs_valid['input_ids']
valid_attention_masks = tokenized_inputs_valid['attention_mask']



In [909]:
train_input_ids.shape, train_attention_masks.shape, valid_input_ids.shape, valid_attention_masks.shape, train_labels.shape, valid_labels.shape

(torch.Size([22278, 178]),
 torch.Size([22278, 178]),
 torch.Size([5570, 178]),
 torch.Size([5570, 178]),
 (22278,),
 (5570,))

In [910]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [911]:
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(valid_labels)  # Add a dimension to match the other tensors

# Create the DataLoader for our training set.
train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=32)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(valid_input_ids, valid_attention_masks, validation_labels)
validation_dataloader = DataLoader(validation_data, shuffle=False, batch_size=32)


In [912]:
mathbert = BertCNNLSTMClassifier.from_pretrained("tbs17/MathBERT", num_labels=df_1['score_to_predict'].nunique()) 

Some weights of the model checkpoint at tbs17/MathBERT were not used when initializing BertCNNLSTMClassifier: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertCNNLSTMClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertCNNLSTMClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [913]:
def train_model(model, epochs, train_dataloader, validation_dataloader):
    # Use GPU, if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    kappa_score_max = 0
    best_model = model

    # Setup optimizer and LR scheduler 
    total_steps = len(train_dataloader) * epochs
    optimizer, scheduler = get_optimizer_and_scheduler(
        model, total_steps, lr=5e-5, weight_decay=0.01)

    loss_values = []
    eval_accs = []
        # Tracking variables
    

    for epoch in range(0, epochs):
        t0 = time.time()

        total_loss = 0
        model.train()
        predictions, true_labels = [], []

        with tqdm(train_dataloader, unit="batch") as train_pbar:
            for batch in train_pbar:
                train_pbar.set_description(f"Training (epoch {epoch + 1})")
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                model.zero_grad()        

              
                outputs = model(
                    input_ids=b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels )
                

                _, loss = outputs


                total_loss += loss.item()


                loss.backward()


                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                optimizer.step()

                # Update the learning rate.
                scheduler.step()

        avg_train_loss = total_loss / len(train_dataloader)            
        
        loss_values.append(avg_train_loss)

        print("  * Average training loss: {0:.2f}".format(avg_train_loss))
        print("  * Training epoch took: {:}".format(format_time(time.time() - t0)))
            
        print("Running Validation...")

        t0 = time.time()
        model.eval()

        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            
            with torch.no_grad():        

                outputs = model(
                    input_ids=b_input_ids, 
                    attention_mask=b_input_mask)
            

            logits = outputs[0]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            # Calculate the accuracy for this batch of test sentences.
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy
            # Track the number of batches
            nb_eval_steps += 1
            predictions.append(logits)
            true_labels.append(label_ids)
       
        predictions = np.concatenate(predictions)
        true_labels = np.concatenate(true_labels)

        predicted_labels = np.argmax(predictions, axis=1)
        kappa_score = cohen_kappa_score(true_labels, predicted_labels, labels=None, weights= 'quadratic', sample_weight=None)
      
        
        
        if kappa_score > kappa_score_max :
          kappa_score_max = kappa_score
          predicted_labels_max = predicted_labels
          best_model = model
        avg_eval_acc = eval_accuracy/nb_eval_steps
        print("  * Accuracy: {0:.2f}".format(avg_eval_acc))
        print("  * Validation took: {:}".format(format_time(time.time() - t0)))
        print("Kappa Score = " + str(kappa_score))
        eval_accs.append(avg_eval_acc)
    print("Training complete!")
    return best_model,kappa_score_max, predicted_labels_max, true_labels

In [914]:
best_model, score, predicted_labels, true_labels = train_model(model=mathbert,epochs=10,train_dataloader=train_dataloader,
  validation_dataloader=validation_dataloader)
print(score)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training (epoch 1): 100%|██████████| 697/697 [02:44<00:00,  4.23batch/s]


  * Average training loss: 0.15
  * Training epoch took: 0:02:45
Running Validation...
  * Accuracy: 0.97
  * Validation took: 0:00:12
Kappa Score = 0.7082160956580059


Training (epoch 2): 100%|██████████| 697/697 [02:44<00:00,  4.23batch/s]


  * Average training loss: 0.09
  * Training epoch took: 0:02:45
Running Validation...
  * Accuracy: 0.97
  * Validation took: 0:00:12
Kappa Score = 0.753325937390962


Training (epoch 3): 100%|██████████| 697/697 [02:45<00:00,  4.21batch/s]


...

  * Average training loss: 0.02
  * Training epoch took: 0:02:45
Running Validation...
  * Accuracy: 0.97
  * Validation took: 0:00:12
Kappa Score = 0.7883385545072015


Training (epoch 9): 100%|██████████| 697/697 [02:44<00:00,  4.23batch/s]


  * Average training loss: 0.01
  * Training epoch took: 0:02:45
Running Validation...
  * Accuracy: 0.97
  * Validation took: 0:00:12
Kappa Score = 0.7680298576038979


Training (epoch 10): 100%|██████████| 697/697 [02:45<00:00,  4.21batch/s]


  * Average training loss: 0.01
  * Training epoch took: 0:02:46
Running Validation...
  * Accuracy: 0.97
  * Validation took: 0:00:12
Kappa Score = 0.7802713334959829
Training complete!
0.7966337926662652


In [915]:
cohen_kappa_score(predicted_labels, true_labels, weights = 'quadratic')

0.7966337926662652

In [916]:
df_result = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/MathBert+CNN/VH271613.csv')
df_result

,indexes,TrueValue,PredictedValue
0,35493,3,3
1,25104,1,1
2,4504,2,1
3,20221,1,1
4,10100,1,1
...,...,...,...
9395,39838,3,3
9396,21563,1,1
9397,39397,3,3
9398,1545,4,3


In [917]:
df_copy = dfs['VH271613'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]
df_copy['indexes'] = df_copy.index
df_copy

,student_id,score_to_predict,srace10,dsex,accom2,iep,lep,indexes
0,00DX05DRh5,1,3.0,2.0,2.0,2.0,2.0,0
1,00N3yEYMAy,1,1.0,2.0,2.0,2.0,2.0,1
2,00TuatdAM2,1,1.0,2.0,2.0,2.0,2.0,2
3,00wXPqjAxM,1,7.0,2.0,2.0,2.0,2.0,3
4,00wtajhH6E,1,2.0,2.0,2.0,2.0,2.0,4
...,...,...,...,...,...,...,...,...
27843,zyUjt9xqkg,1,3.0,1.0,1.0,2.0,1.0,27843
27844,zzBMPZ2ulP,1,3.0,1.0,2.0,2.0,1.0,27844
27845,zzEWXuRyL6,1,1.0,1.0,2.0,2.0,2.0,27845
27846,zzfwDRPzZC,1,2.0,1.0,2.0,2.0,2.0,27846


In [918]:
merged_df = df_result.merge(df_copy, on="indexes", how="inner")

merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH271613_mathbert.csv', index = False)
merged_df

,indexes,TrueValue,PredictedValue,student_id,score_to_predict,srace10,dsex,accom2,iep,lep
0,25104,1,1,lQNKRRISIF,1,3.0,1.0,2.0,2.0,2.0
1,4504,2,1,H6x8ZWZGLm,1,2.0,1.0,2.0,2.0,2.0
2,20221,1,1,KoNTiAblh9,1,3.0,2.0,2.0,2.0,2.0
3,10100,1,1,cYkbh1UiQD,1,1.0,2.0,2.0,2.0,2.0
4,27229,3,3,whnXahZoTI,1,3.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
5554,17471,2,1,6KjQlsYqpI,1,2.0,1.0,1.0,1.0,2.0
5555,13422,2,1,pCgcDx48Vr,1,2.0,1.0,2.0,2.0,2.0
5556,9424,1,1,ZzweoWqnvO,3,1.0,2.0,2.0,2.0,2.0
5557,21563,1,1,S6rzbgbn8Z,1,2.0,2.0,2.0,2.0,2.0


In [919]:
test_indexes = df_test['student_id']
score = df_test['score_to_predict']

In [923]:
result_VH271613_mathbert = pd.DataFrame()
result_VH271613_mathbert['student_id'] = test_indexes
result_VH271613_mathbert['score'] = score
result_VH271613_mathbert['true_labels'] = true_labels +1 
result_VH271613_mathbert['predicted'] = predicted_labels + 1

In [924]:
df_copy = dfs['VH271613'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [ ]:
df_copy

,student_id,score_to_predict,srace10,dsex,accom2,iep,lep
0,00DX05DRh5,1,3.0,2.0,2.0,2.0,2.0
1,00N3yEYMAy,1,1.0,2.0,2.0,2.0,2.0
2,00TuatdAM2,1,1.0,2.0,2.0,2.0,2.0
3,00wXPqjAxM,1,7.0,2.0,2.0,2.0,2.0
4,00xmXONOc1,1,3.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...
26667,zxmwVPA9NG,1,2.0,1.0,2.0,2.0,2.0
26668,zyUjt9xqkg,1,3.0,1.0,1.0,2.0,1.0
26669,zzBMPZ2ulP,1,3.0,1.0,2.0,2.0,1.0
26670,zzEWXuRyL6,1,1.0,1.0,2.0,2.0,2.0


In [925]:
merged_df = result_VH271613_mathbert.merge(df_copy, on="student_id", how="inner")

merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH271613_mathbert.csv', index = False)
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,dVO88C3Wzi,1,1,1,1,2.0,2.0,2.0,2.0,2.0
1,EWsdrxJz6j,1,1,1,1,3.0,1.0,1.0,2.0,1.0
2,Ljt31oskkx,1,1,1,1,1.0,2.0,2.0,2.0,2.0
3,DnKR6gdrV4,1,1,1,1,7.0,2.0,2.0,2.0,2.0
4,L6sIJG1dK5,1,1,1,1,1.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...
5565,uSTitYH4LS,1,1,1,1,7.0,2.0,2.0,2.0,2.0
5566,trTZRU84Ex,1,1,1,1,2.0,1.0,2.0,2.0,2.0
5567,7YwSZsGTmk,1,1,1,1,3.0,2.0,1.0,2.0,1.0
5568,5gViUV3ZSs,1,1,1,1,1.0,2.0,2.0,2.0,2.0


### Test Predictions



In [926]:
df_test = dfs_test['VH271613']
df_test["predict_from"]= df_test["predict_from"].fillna('N/A')
df_1['predict_from'] = df_1["predict_from"].apply(contractions.fix)
df_test


,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
14247,00u9rrqZuh,VH271613,1,it seys in the tope Phil's age will be 3 times...,2017,3,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c(""1"", """")",list()
14248,02G1Gxbmhf,VH271613,1,zach is younger than Alexs is 6time older than...,2017,2,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
14249,053OYb9O4y,VH271613,1,i dont know,2017,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c("""", ""2"")",list()
14250,05M1oLdPvU,VH271613,1,cause it did not tell you their age,2017,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""1"")","c(NA, 2)",list()
14251,06e1PZnECl,VH271613,1,THE STATEMENT IS NOT TRUE BECAUSE PHIL IS 3 TI...,2017,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17336,zrKGO3SIy8,VH271613,1,56,2019,2,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 1,TRUE FALSE,FALSE FALSE
17337,zutovXyxK3,VH271613,1,tim is six years younger then phile so that te...,2019,6,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 2,TRUE FALSE,FALSE FALSE
17338,zw8QXgpLnG,VH271613,1,Because the amount of how much older you are t...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE
17339,zyyahLMxtS,VH271613,1,Because Zach is already one year younger then ...,2019,7,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE


In [927]:
df_test['predict_from'].isna().sum()

0

In [928]:
test_inputs = df_test.predict_from.values.tolist()

In [929]:
tokenized_inputs_test = meth_bert_tokenizer(test_inputs, add_special_tokens=True,
        padding='max_length',
        max_length=64,
        return_tensors='pt',
        truncation=True)

In [930]:
test_input_ids = tokenized_inputs_test['input_ids']
test_attention_masks = tokenized_inputs_test['attention_mask']

In [931]:
test_input_ids.shape

torch.Size([3094, 64])

In [932]:
test_data = TensorDataset(test_input_ids, test_attention_masks)
test_loader = DataLoader(test_data, shuffle=True, batch_size=32)

In [933]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [934]:
t0 = time.time()
predictions = []
best_model.eval()
for batch in test_loader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch  # Remove the unused b_labels
    
    with torch.no_grad():
        outputs = best_model(input_ids=b_input_ids, attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)

predictions = np.concatenate(predictions)
predicted_labels = np.argmax(predictions, axis=1) + 1
print(np.unique(predicted_labels))

[1 2 3]


In [935]:
len(predicted_labels)

3094

In [936]:
print(np.unique(predicted_labels, return_counts=True))

(array([1, 2, 3]), array([2955,   39,  100]))


In [937]:
df_test['score_to_predict'] = predicted_labels
df_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH271613.csv', index = False)
df_test

,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
14247,00u9rrqZuh,VH271613,1,it seys in the tope Phil's age will be 3 times...,2017,3,1.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c(""1"", """")",list()
14248,02G1Gxbmhf,VH271613,1,zach is younger than Alexs is 6time older than...,2017,2,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
14249,053OYb9O4y,VH271613,1,i dont know,2017,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""2"")","c("""", ""2"")",list()
14250,05M1oLdPvU,VH271613,1,cause it did not tell you their age,2017,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""2"", ""1"")","c(NA, 2)",list()
14251,06e1PZnECl,VH271613,1,THE STATEMENT IS NOT TRUE BECAUSE PHIL IS 3 TI...,2017,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c(""1"", ""1"")","c(""1"", """")",list()
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17336,zrKGO3SIy8,VH271613,1,56,2019,2,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 1,TRUE FALSE,FALSE FALSE
17337,zutovXyxK3,VH271613,1,tim is six years younger then phile so that te...,2019,6,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 2,TRUE FALSE,FALSE FALSE
17338,zw8QXgpLnG,VH271613,1,Because the amount of how much older you are t...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE
17339,zyyahLMxtS,VH271613,1,Because Zach is already one year younger then ...,2019,7,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 2,FALSE TRUE,FALSE FALSE


### Electra


In [825]:
MODEL = "google/electra-base-discriminator"

In [851]:
df_1 = dfs['VH271613']
df_1_test = dfs_test['VH271613']

In [852]:
df_1 = df_1.dropna(subset = ['predict_from'])

In [853]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]]
df_1['labels'] = df_1['score_to_predict'] - 1
#df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
#df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

<ipython-input-853-14cb78c66525>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['labels'] = df_1['score_to_predict'] - 1


,student_id,predict_from,score_to_predict,labels
0,00DX05DRh5,it is not true because phil's is 3 times yonge...,1,0
1,00N3yEYMAy,because is zach is 4 years older than tim that...,1,0
2,00TuatdAM2,because now zach will be older than tim and alex,1,0
3,00wXPqjAxM,"because we older ,not yonger",1,0
4,00wtajhH6E,because i,1,0


In [837]:
df_train, df_valid = train_test_split(df_1, test_size = 0.2, stratify = df_1['labels'], random_state=11 )

In [854]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True,  add_special_tokens = True, max_length = 200)   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])    

In [855]:
dataset_train = d1.from_pandas(df_train, preserve_index=False)
dataset_valid = d1.from_pandas(df_valid, preserve_index=False)
model = ElectraForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
model.cuda()
tokenizer = ElectraTokenizerFast.from_pretrained(MODEL, do_lower_case=True)
train_dataset = TrainerDataset(dataset_train["predict_from"],
                              dataset_train["labels"], tokenizer)
eval_dataset = TrainerDataset(dataset_valid["predict_from"],
                            dataset_valid["labels"], tokenizer) 

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

In [856]:
training_args = TrainingArguments(
  output_dir="./models/model_electra2_prompt_4",
  num_train_epochs=4,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=96,
  per_device_eval_batch_size=32,
  save_total_limit = 2,
  save_strategy = 'epoch',
  load_best_model_at_end=False ) # Make sure all batches are of equal size)
# Instantiate the Trainer class
trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Cohen Kappa
1,No log,0.077789,0.739761
2,No log,0.067764,0.770289
3,0.103300,0.064752,0.783770
4,0.103300,0.066446,0.784237


TrainOutput(global_step=932, training_loss=0.08125369231588339, metrics={'train_runtime': 387.3928, 'train_samples_per_second': 230.03, 'train_steps_per_second': 2.406, 'total_flos': 4762583085117312.0, 'train_loss': 0.08125369231588339, 'epoch': 4.0})

In [ ]:
pred, actual, _ = trainer.predict(eval_dataset)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.941512269783561

In [ ]:
test_indexes = dataset_valid['student_id']
score = dataset_valid['score_to_predict']

In [ ]:
result_VH302907_electra = pd.DataFrame()
result_VH302907_electra['student_id'] = test_indexes
result_VH302907_electra['score'] = score
result_VH302907_electra['true_labels'] = actual +1 
result_VH302907_electra['predicted'] = pred_labels + 1

In [ ]:
df_copy = dfs['VH302907'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [ ]:
merged_df = result_VH302907_electra.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,HhIIQhMOqd,2,2,2,2,3.0,2.0,2.0,2.0,2.0
1,4rKTKky2Sz,2,2,2,2,1.0,2.0,2.0,1.0,2.0
2,ZrUNbAs5eZ,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3,A5iA75aQy3,2,2,2,2,7.0,1.0,2.0,2.0,2.0
4,5Gnf05ZbpS,1,1,1,1,1.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...
7630,L5poVFl9FB,1,1,1,1,2.0,2.0,2.0,2.0,2.0
7631,TQ0oX1TE8S,2,2,2,2,3.0,1.0,2.0,2.0,2.0
7632,YFSaEROH1x,1,1,1,1,3.0,2.0,2.0,2.0,2.0
7633,QJz8ta4N8U,1,1,1,1,1.0,1.0,2.0,1.0,2.0


In [ ]:
merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH302907_electra.csv', index = False)

###Test Prediction

In [ ]:
class TestDataset(Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, max_length = 100, add_special_tokens = True)
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],)   

In [ ]:
df_1_test['predict_from'] = df_1_test['predict_from'].fillna(df_1_test['parsed_xml_v1'])
df_1_test['predict_from'] = df_1_test['predict_from'].fillna('N/A')
df_1_test['predict_from'].isna().sum()

0

In [ ]:
df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
dataset_test = d1.from_pandas(df_1_test, preserve_index=False)
test_dataset = TestDataset(dataset_test["predict_from"], tokenizer) 



<ipython-input-491-0d545356f869>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
<ipython-input-491-0d545356f869>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)


In [ ]:
pred ,_,_ = trainer.predict(test_dataset)

In [ ]:
pred_labels = np.argmax(pred, axis=1)

In [ ]:
pred_labels

array([0, 0, 0, ..., 1, 1, 0])

In [ ]:
df_1_test['score_to_predict'] = pred_labels + 1

In [ ]:
df_1_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH302907.csv', index = False)

## Prompt 7 : VH302907

In [475]:
MODEL = "google/electra-base-discriminator"

In [476]:
df_1 = dfs['VH302907']
df_1_test = dfs_test['VH302907']

In [ ]:
df_1 = df_1.dropna(subset = ['predict_from'])

In [477]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]]
df_1['labels'] = df_1['score_to_predict'] - 1
df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

<ipython-input-477-72af636fc434>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['labels'] = df_1['score_to_predict'] - 1
<ipython-input-477-72af636fc434>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
<ipython-input-477-72af636fc434>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

,student_id,predict_from,score_to_predict,labels
0,00DBhQakLo,108,1,0
1,00LTCar9r2,it is 540 because if you take 5 times 108 you ...,1,0
2,00Zam1QWAQ,52180,2,1
3,00dI84ZX78,all the sides are the same length and they are...,1,0
4,00e1jyrmzR,180,1,0


In [478]:
df_train, df_valid = train_test_split(df_1, test_size = 0.2, stratify = df_1['labels'], random_state=11 )

In [480]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, max_length = 100, add_special_tokens = True)   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])    

In [481]:
dataset_train = d1.from_pandas(df_train, preserve_index=False)
dataset_valid = d1.from_pandas(df_valid, preserve_index=False)
model = ElectraForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
model.cuda()
tokenizer = ElectraTokenizerFast.from_pretrained(MODEL, do_lower_case=True)
train_dataset = TrainerDataset(dataset_train["predict_from"],
                              dataset_train["labels"], tokenizer)
eval_dataset = TrainerDataset(dataset_valid["predict_from"],
                            dataset_valid["labels"], tokenizer) 

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

In [482]:
training_args = TrainingArguments(
  output_dir="./models/model_electra2_prompt_4",
  num_train_epochs=5,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  save_total_limit = 2,
  save_strategy = 'epoch',
  load_best_model_at_end=False ) # Make sure all batches are of equal size)
# Instantiate the Trainer class
trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Cohen Kappa
1,0.152000,0.099821,0.926036
2,0.092000,0.075863,0.935758
3,0.077200,0.071092,0.936824
4,0.058500,0.054989,0.939152
5,0.048300,0.056372,0.941512


TrainOutput(global_step=4775, training_loss=0.08240090215393386, metrics={'train_runtime': 738.8881, 'train_samples_per_second': 206.648, 'train_steps_per_second': 6.462, 'total_flos': 7846567781820000.0, 'train_loss': 0.08240090215393386, 'epoch': 5.0})

In [483]:
pred, actual, _ = trainer.predict(eval_dataset)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.941512269783561

In [484]:
test_indexes = dataset_valid['student_id']
score = dataset_valid['score_to_predict']

In [485]:
result_VH302907_electra = pd.DataFrame()
result_VH302907_electra['student_id'] = test_indexes
result_VH302907_electra['score'] = score
result_VH302907_electra['true_labels'] = actual +1 
result_VH302907_electra['predicted'] = pred_labels + 1

In [486]:
df_copy = dfs['VH302907'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [487]:
merged_df = result_VH302907_electra.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,HhIIQhMOqd,2,2,2,2,3.0,2.0,2.0,2.0,2.0
1,4rKTKky2Sz,2,2,2,2,1.0,2.0,2.0,1.0,2.0
2,ZrUNbAs5eZ,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3,A5iA75aQy3,2,2,2,2,7.0,1.0,2.0,2.0,2.0
4,5Gnf05ZbpS,1,1,1,1,1.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...
7630,L5poVFl9FB,1,1,1,1,2.0,2.0,2.0,2.0,2.0
7631,TQ0oX1TE8S,2,2,2,2,3.0,1.0,2.0,2.0,2.0
7632,YFSaEROH1x,1,1,1,1,3.0,2.0,2.0,2.0,2.0
7633,QJz8ta4N8U,1,1,1,1,1.0,1.0,2.0,1.0,2.0


In [488]:
merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH302907_electra.csv', index = False)

###Test Prediction

In [489]:
class TestDataset(Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, max_length = 100, add_special_tokens = True)
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],)   

In [490]:
df_1_test['predict_from'] = df_1_test['predict_from'].fillna(df_1_test['parsed_xml_v1'])
df_1_test['predict_from'] = df_1_test['predict_from'].fillna('N/A')
df_1_test['predict_from'].isna().sum()

0

In [491]:
df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
dataset_test = d1.from_pandas(df_1_test, preserve_index=False)
test_dataset = TestDataset(dataset_test["predict_from"], tokenizer) 



<ipython-input-491-0d545356f869>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
<ipython-input-491-0d545356f869>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)


In [492]:
pred ,_,_ = trainer.predict(test_dataset)

In [493]:
pred_labels = np.argmax(pred, axis=1)

In [ ]:
pred_labels

array([0, 0, 0, ..., 1, 1, 0])

In [494]:
df_1_test['score_to_predict'] = pred_labels + 1

In [495]:
df_1_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH302907.csv', index = False)

In [496]:
df_1_test

,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
17341,01R1SshJRr,VH302907,1,28,2017,3,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17342,02VUMWgF0u,VH302907,1,The sum of the interior angles is 540 degrees ...,2017,3,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17343,02gJj5RXOQ,VH302907,1,el 5 es muy pequeno de 540,2017,3,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17344,05DxgErGjq,VH302907,1,because they all abtus angles and they have to...,2017,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17345,0APbHollMO,VH302907,1,50,2017,1,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21577,zjKSoxMKLn,VH302907,1,Because all of them are obtuse.,2019,2,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21578,zq0O7g1kXG,VH302907,1,because the numbebrs of the angles add up to 540.,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21579,zwOGtkboto,VH302907,2,"540÷3=180,180+180+180=540",2019,3,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21580,zy5s8FDzc1,VH302907,1,It's the sum because you have to use the equat...,2019,4,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Prompt 8 : VH304954

In [1]:
MODEL = "distilbert-base-uncased"

In [14]:
df_1  = dfs['VH304954']
df_test = dfs_test['VH304954']

In [15]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]].fillna("")
df_1['labels'] = df_1['score_to_predict'] - 1
df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

,student_id,predict_from,score_to_predict,labels
0,00H9cLjFgx,14348,2,1
1,00OSoOY4af,mark needs to do next is to add 5 to the answe...,1,0
2,00UGtYZO4Y,he needs to take away 43 48,1,0
3,00lNNHto40,5,3,2
4,00mbt1B9QC,what he needs to do next is subtract 5 from 100,3,2


In [16]:
train_df, test_df = train_test_split(df_1, test_size=0.2, random_state=11, stratify = df_1['labels'])
train_dataset = d1.from_pandas(train_df, preserve_index=True)
test_dataset = d1.from_pandas(test_df, preserve_index=True)

In [17]:
df_1.shape,test_df.shape

((24686, 4), (4938, 4))

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
tokenizer = AutoTokenizer.from_pretrained(MODEL,  do_lower_case=True)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [20]:
model.resize_token_embeddings(len(tokenizer))

def tokenize_function(examples):
    #examples["predict_from"] = ["[CLS] " + text + " [SEP]" for text in examples["predict_from"]]
    return tokenizer(examples["predict_from"],padding=True,add_special_tokens = True)



In [21]:
train_datasets = train_dataset.map(tokenize_function, batched=True)
test_datasets = test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/19748 [00:00<?, ? examples/s]

Map:   0%|          | 0/4938 [00:00<?, ? examples/s]

In [ ]:
train_datasets, test_datasets

(Dataset({
     features: ['student_id', 'predict_from', 'score_to_predict', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
     num_rows: 15644
 }),
 Dataset({
     features: ['student_id', 'predict_from', 'score_to_predict', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
     num_rows: 3911
 }))

In [23]:
training_args = TrainingArguments(output_dir="./models/model_electra2_prompt_4",
  num_train_epochs=5,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  save_total_limit = 2,
  save_strategy = 'epoch',
  load_best_model_at_end=False
                                  )


trainer = Trainer(model=model,args=training_args,train_dataset=train_datasets,  eval_dataset=test_datasets,
                  compute_metrics=compute_metrics)
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 15>:15                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1899 in _inner_training_loop     │
│                                                                                                  │
│   1896 │   │   │   │   rng_to_sync = True                                                        │
│   1897 │   │   │                                                                                 │
│   1898 │   │   │   step = -1                                                                     │
│ ❱ 1899 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1900 │   │   │   │   total_batched_samples += 1                                                │
│   1901 │   │   │   │   if rng_to_sync:                                                           │
│   1902 │   │   │   │   │   self._load_rng_state(resume_from_checkpoint)                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:633 in __next__           │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                                                      │
│    634 │   │   │   self._num_yielded += 1                                                        │
│    635 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    636 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:677 in _next_data         │
│                                                                                                  │
│    674 │                                                                                         │
│    675 │   def _next_data(self):                                                                 │
│    676 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  677 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    678 │   │   if self._pin_memory:                        

In [505]:
df_results = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/DistillBert/VH304954.csv')

In [506]:
df_results = df_results.drop(['Unnamed: 0'], axis = 1)
# Rename the column name "_1" to "student_id"
df_results = df_results.rename(columns={"indexes": "student_id", 'True Labels' : 'true_labels', 'PredictedValue': "predicted"})
df_results

,student_id,true_labels,predicted
0,6EzyAYNIvF,1,2
1,2V73IkCNb6,1,2
2,zJOZdz7Pyp,2,2
3,4UWcZCKdzL,3,3
4,k8soIBNYdU,1,1
...,...,...,...
1951,Mv0aESvONm,1,2
1952,tPbdj7y9fG,3,3
1953,Gss4WtBdiP,1,1
1954,4cH74RJqJd,2,2


In [510]:
pred, actual, _ = trainer.predict(test_datasets)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.7140413118432161

In [511]:
test_indexes = test_dataset['student_id']
score = test_dataset['score_to_predict']

In [512]:
result_VH139380_distill = pd.DataFrame()
result_VH139380_distill['student_id'] = test_indexes
result_VH139380_distill['score'] = score
result_VH139380_distill['true_labels'] = actual +1 
result_VH139380_distill['predicted'] = pred_labels + 1

In [513]:
df_copy = dfs['VH304954'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [518]:
merged_df = result_VH139380_distill.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,5bRnrnAeO3,3,3,3,3,4.0,1.0,NaN,2.0,2.0
1,fHLXk9Kjhv,2,2,2,2,1.0,2.0,NaN,2.0,2.0
2,yEofMLbyzd,1,1,1,1,2.0,2.0,NaN,2.0,2.0
3,oXVjZluvlK,3,3,3,3,4.0,1.0,NaN,2.0,2.0
4,EHfpeWYecx,2,2,2,2,2.0,2.0,NaN,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
4933,hMLmcvqgBD,2,2,2,2,2.0,2.0,NaN,2.0,2.0
4934,43aktpm4j3,2,2,2,2,2.0,1.0,NaN,2.0,2.0
4935,E1khcd0xhN,1,1,1,1,1.0,1.0,NaN,2.0,2.0
4936,ZHMTVFDoDq,1,1,2,1,3.0,2.0,NaN,2.0,2.0


In [519]:

merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH304954_distill.csv', index = False)
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,5bRnrnAeO3,3,3,3,3,4.0,1.0,NaN,2.0,2.0
1,fHLXk9Kjhv,2,2,2,2,1.0,2.0,NaN,2.0,2.0
2,yEofMLbyzd,1,1,1,1,2.0,2.0,NaN,2.0,2.0
3,oXVjZluvlK,3,3,3,3,4.0,1.0,NaN,2.0,2.0
4,EHfpeWYecx,2,2,2,2,2.0,2.0,NaN,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
4933,hMLmcvqgBD,2,2,2,2,2.0,2.0,NaN,2.0,2.0
4934,43aktpm4j3,2,2,2,2,2.0,1.0,NaN,2.0,2.0
4935,E1khcd0xhN,1,1,1,1,1.0,1.0,NaN,2.0,2.0
4936,ZHMTVFDoDq,1,1,2,1,3.0,2.0,NaN,2.0,2.0


In [528]:
true_labels = merged_df['true_labels']
pred = merged_df['predicted']

In [529]:
cohen_kappa_score(pred, true_labels, weights = 'quadratic')

0.7140413118432162

### Test Predictions

In [530]:
test_dataset = d1.from_pandas(df_test, preserve_index=False)

In [531]:
test_datasets = test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/2743 [00:00<?, ? examples/s]

In [532]:
test_datasets

Dataset({
    features: ['student_id', 'accession', 'score_to_predict', 'predict_from', 'year', 'srace10', 'dsex', 'accom2', 'iep', 'lep', 'rater_1', 'pta_rtr1', 'ptb_rtr1', 'ptc_rtr1', 'composite', 'score', 'assigned_score', 'ee_use', 'parsed_xml_v1', 'parsed_xml_v2', 'parsed_xml_v3', 'source1', 'source2', 'source3', 'source4', 'target1', 'target2', 'target3', 'target4', 'eliminations', 'selected', 'eliminated', 'selected1', 'selected2', 'selected3', 'selected4', 'eliminated1', 'eliminated2', 'eliminated3', 'eliminated4', 'selected1.1', 'selected2.1', 'eliminated1.1', 'eliminated2.1', 'partA_response_val', 'partB_response_val', 'partB_eliminations', 'input_ids', 'attention_mask'],
    num_rows: 2743
})

In [533]:
pred, _, _ = trainer.predict(test_datasets)
pred_labels = np.argmax(pred, axis=1)
pred_labels

array([0, 1, 0, ..., 0, 0, 1])

In [534]:
df_test_score = df_test.copy()
df_test_score['score_to_predict'] = pred_labels + 1
df_test_score['score_to_predict'].value_counts()

2    1315
1     845
3     583
Name: score_to_predict, dtype: int64

In [535]:
df_test_score.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH304954.csv', index = False)

## Prompt 9 : VH507804

In [536]:
MODEL = "google/electra-base-discriminator"

In [560]:
df_1 = dfs['VH507804']
df_1_test = dfs_test['VH507804']

In [561]:
df_1 = df_1.dropna(subset = ['predict_from'])

In [562]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]]
df_1['labels'] = df_1['score_to_predict'] - 1
#df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
#df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

<ipython-input-562-14cb78c66525>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['labels'] = df_1['score_to_predict'] - 1


,student_id,predict_from,score_to_predict,labels
0,000qML6Slq,efsddsf,1,0
1,00BRxCIEhJ,Multiply the largest numbers by each other the...,3,2
2,00XUIxWsfr,37+17=54,1,0
3,00aiAJIsWf,Greatest to lowest.,1,0
4,00jAL26DNW,64×54-17,1,0


In [563]:
df_train, df_valid = train_test_split(df_1, test_size = 0.2, stratify = df_1['labels'], random_state=11 )

In [564]:
df_valid.shape

(3235, 4)

In [565]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        #inputs = ["[CLS] " + text + " [SEP]" for text in inputs]
        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, max_length = 64, add_special_tokens = True)   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])    

In [566]:
dataset_train = d1.from_pandas(df_train, preserve_index=False)
dataset_valid = d1.from_pandas(df_valid, preserve_index=False)
model = ElectraForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
model.cuda()
tokenizer = ElectraTokenizerFast.from_pretrained(MODEL, do_lower_case=True)
train_dataset = TrainerDataset(dataset_train["predict_from"],
                              dataset_train["labels"], tokenizer)
eval_dataset = TrainerDataset(dataset_valid["predict_from"],
                            dataset_valid["labels"], tokenizer) 

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.o

In [567]:
training_args = TrainingArguments(
  output_dir="./models/model_electra2_prompt_4",
  num_train_epochs=5,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  save_total_limit = 2,
  save_strategy = 'epoch',
  load_best_model_at_end=False ) # Make sure all batches are of equal size)
# Instantiate the Trainer class
trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Cohen Kappa
1,No log,0.216836,0.894559
2,0.269800,0.180023,0.922368
3,0.146000,0.187777,0.933539
4,0.097400,0.187379,0.936097
5,0.064400,0.193658,0.937613


TrainOutput(global_step=2025, training_loss=0.143807856241862, metrics={'train_runtime': 227.0544, 'train_samples_per_second': 284.932, 'train_steps_per_second': 8.919, 'total_flos': 2127765319989120.0, 'train_loss': 0.143807856241862, 'epoch': 5.0})

In [568]:
pred, actual, _ = trainer.predict(eval_dataset)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.9376125094887784

In [569]:
test_indexes = dataset_valid['student_id']
score = dataset_valid['score_to_predict']

In [570]:
result_VH507804_electra = pd.DataFrame()
result_VH507804_electra['student_id'] = test_indexes
result_VH507804_electra['score'] = score
result_VH507804_electra['true_labels'] = actual +1 
result_VH507804_electra['predicted'] = pred_labels + 1

In [571]:
df_copy = dfs['VH507804'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [572]:
merged_df = result_VH507804_electra.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,nB6mhCmqXz,1,1,1,1,1.0,2.0,2.0,2.0,2.0
1,v2CNItgTcj,1,1,1,1,3.0,2.0,2.0,2.0,1.0
2,T0zCPZfe75,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3,uFDXTQ9Ej7,1,1,1,1,1.0,1.0,2.0,2.0,2.0
4,gaINsa9O0t,3,3,3,3,3.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
3230,LHpmv0UVyR,1,1,1,1,2.0,2.0,1.0,1.0,2.0
3231,an1cKrrBqk,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3232,JePa9BD8Mw,1,1,1,1,1.0,1.0,1.0,1.0,2.0
3233,tKg2sYtVVI,1,1,1,1,3.0,2.0,1.0,2.0,1.0


In [573]:
merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH507804_electra.csv', index = False)

###Test Prediction

In [ ]:
class TestDataset(Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

        inputs = ["[CLS] " + text + " [SEP]" for text in inputs]
        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, max_length = 80, add_special_tokens = True)
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],)   

In [ ]:
df_1_test['predict_from'] = df_1_test['predict_from'].fillna(df_1_test['parsed_xml_v1'])
df_1_test['predict_from'] = df_1_test['predict_from'].fillna('N/A')
df_1_test['predict_from'].isna().sum()

0

In [ ]:
df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
dataset_test = d1.from_pandas(df_1_test, preserve_index=False)
test_dataset = TestDataset(dataset_test["predict_from"], tokenizer) 



In [ ]:
pred ,_,_ = trainer.predict(test_dataset)

In [ ]:
pred_labels = np.argmax(pred, axis=1)

In [ ]:
pred_labels

array([0, 0, 0, ..., 0, 2, 0])

In [ ]:
df_1_test['score_to_predict'] = pred_labels + 1

In [ ]:
df_1_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH507804.csv', index = False)

In [ ]:
df_1_test

,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
24325,04NoFSBa8L,VH507804,1,54,2019,1,2.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24326,06cgyZgZwD,VH507804,1,she should have put it in the second box.,2019,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24327,0CmVH4sCgo,VH507804,1,62×54-17,2019,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24328,0DotTnLvcL,VH507804,1,the rule is you do it less to gratest,2019,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24329,0FebHNmRyB,VH507804,3,Multiply the largest numbers then add the seco...,2019,7,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26117,zvc2Tri4uV,VH507804,1,45,2019,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26118,zvmTzDMVLu,VH507804,1,the larget result is 595939,2019,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26119,zxnuVq3WlV,VH507804,1,62,2019,3,2.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26120,zxtnowkKPB,VH507804,3,Get the largest numbers and mutipliy them then...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Prompt 10 : VH525628

In [32]:
MODEL = "google/electra-large-discriminator"

In [38]:
df_1 = dfs['VH525628']
df_1_test = dfs_test['VH525628']

In [39]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]]
df_1['labels'] = df_1['score_to_predict'] - 1
df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

<ipython-input-39-72af636fc434>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['labels'] = df_1['score_to_predict'] - 1
<ipython-input-39-72af636fc434>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
<ipython-input-39-72af636fc434>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

,student_id,predict_from,score_to_predict,labels
0,00ONGl5PbU,i believe i am right because if you get the bi...,1,0
1,00Pyb7DQQJ,i know this might be right because of the x an...,1,0
2,00mk2jVZeW,since z is the smallest and w the biggest you ...,3,2
3,00oZfN72Ey,i know because x always comes first then z is ...,1,0
4,016tN2x0ht,i know this answer is correct because if you a...,2,1


In [40]:
df_train, df_valid = train_test_split(df_1, test_size = 0.2, stratify = df_1['labels'], random_state=11 )

In [41]:
df_valid.shape

(3255, 4)

In [42]:
df_1['labels'].value_counts()

0    11659
1     3094
2     1522
Name: labels, dtype: int64

In [43]:
class TrainerDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True,  add_special_tokens = True)   

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],
            label=self.targets[idx])    

In [44]:
dataset_train = d1.from_pandas(df_train, preserve_index=False)
dataset_valid = d1.from_pandas(df_valid, preserve_index=False)
model = ElectraForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
model.cuda()
tokenizer = ElectraTokenizerFast.from_pretrained(MODEL, do_lower_case=True)
train_dataset = TrainerDataset(dataset_train["predict_from"],
                              dataset_train["labels"], tokenizer)
eval_dataset = TrainerDataset(dataset_valid["predict_from"],
                            dataset_valid["labels"], tokenizer) 

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier

In [48]:
training_args = TrainingArguments(
  output_dir="./models/model_electra2_prompt_4",
  num_train_epochs=5,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=150,
  per_device_eval_batch_size=112,
  save_total_limit = 2,
  save_strategy = 'epoch',
  load_best_model_at_end=False ) # Make sure all batches are of equal size)
# Instantiate the Trainer class
trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics)
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 18>:18                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1929 in _inner_training_loop     │
│                                                                                                  │
│   1926 │   │   │   │   │   with model.no_sync():                                                 │
│   1927 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1928 │   │   │   │   else:                                                                     │
│ ❱ 1929 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1930 │   │   │   │                                                                             │
│   1931 │   │   │   │   if (                                                                      │
│   1932 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2699 in training_step            │
│                                                                                                  │
│   2696 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2697 │   │                                                                                     │
│   2698 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2699 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2700 │   │                                                                                     │
│   2701 │   │   if self.args.n_gpu > 1:                                                           │
│   2702 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2731 in compute_loss             │
│                                                                                                  │
│   2728 │   │   │   labels = inputs.pop("labels")                                                 │
│   2729 │   │   else:                                                                             │
│   2730 │   │   │   labels = None                                                                 │
│ ❱ 2731 │   │   outputs = model(**inputs)                                                         │
│   2732 │   │   # Save past state if it exists              

In [ ]:
pred, actual, _ = trainer.predict(eval_dataset)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.9503421358930954

In [ ]:
test_indexes = dataset_valid['student_id']
score = dataset_valid['score_to_predict']

In [ ]:
result_VH507804_electra = pd.DataFrame()
result_VH507804_electra['student_id'] = test_indexes
result_VH507804_electra['score'] = score
result_VH507804_electra['true_labels'] = actual +1 
result_VH507804_electra['predicted'] = pred_labels + 1

In [ ]:
df_copy = dfs['VH507804'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [ ]:
merged_df = result_VH507804_electra.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,OC2DZGYeBt,1,1,1,1,5.0,1.0,2.0,2.0,2.0
1,xRw5OVDhoj,1,1,1,1,4.0,2.0,2.0,2.0,1.0
2,wtotYOSvqU,1,1,1,1,2.0,1.0,2.0,1.0,2.0
3,Oea4XDtmoo,1,1,1,1,2.0,2.0,2.0,2.0,2.0
4,3pFfk0qbS3,1,1,1,1,1.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...
2422,eoFFj9vSKb,1,1,1,1,3.0,2.0,2.0,2.0,1.0
2423,ujjJOZ9zmB,1,1,1,1,3.0,2.0,2.0,2.0,2.0
2424,xWSqcyUfVg,2,2,1,2,1.0,2.0,2.0,2.0,2.0
2425,AdXadB9Qe5,1,1,1,1,5.0,1.0,2.0,2.0,2.0


In [ ]:
merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH507804_electra.csv', index = False)

###Test Prediction

In [ ]:
class TestDataset(Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

        inputs = ["[CLS] " + text + " [SEP]" for text in inputs]
        # Tokenize the input
        self.tokenized_inputs = tokenizer(inputs, padding=True, 
                                          truncation=True, max_length = 80, add_special_tokens = True)
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return InputFeatures(
            input_ids=self.tokenized_inputs['input_ids'][idx],
            token_type_ids=self.tokenized_inputs['token_type_ids'][idx],
            attention_mask=self.tokenized_inputs['attention_mask'][idx],)   

In [ ]:
df_1_test['predict_from'] = df_1_test['predict_from'].fillna(df_1_test['parsed_xml_v1'])
df_1_test['predict_from'] = df_1_test['predict_from'].fillna('N/A')
df_1_test['predict_from'].isna().sum()

0

In [ ]:
df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].fillna('N/A')
df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
dataset_test = d1.from_pandas(df_1_test, preserve_index=False)
test_dataset = TestDataset(dataset_test["predict_from"], tokenizer) 



In [ ]:
pred ,_,_ = trainer.predict(test_dataset)

In [ ]:
pred_labels = np.argmax(pred, axis=1)

In [ ]:
pred_labels

array([0, 0, 0, ..., 0, 2, 0])

In [ ]:
df_1_test['score_to_predict'] = pred_labels + 1

In [ ]:
df_1_test.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH507804.csv', index = False)

In [ ]:
df_1_test

,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
24325,04NoFSBa8L,VH507804,1,54,2019,1,2.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24326,06cgyZgZwD,VH507804,1,she should have put it in the second box.,2019,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24327,0CmVH4sCgo,VH507804,1,62×54-17,2019,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24328,0DotTnLvcL,VH507804,1,the rule is you do it less to gratest,2019,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24329,0FebHNmRyB,VH507804,3,Multiply the largest numbers then add the seco...,2019,7,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26117,zvc2Tri4uV,VH507804,1,45,2019,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26118,zvmTzDMVLu,VH507804,1,the larget result is 595939,2019,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26119,zxnuVq3WlV,VH507804,1,62,2019,3,2.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26120,zxtnowkKPB,VH507804,3,Get the largest numbers and mutipliy them then...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Prompt 10 : VH525628 (Distill)

In [616]:
MODEL = "distilbert-base-uncased"

In [701]:
df_1  = dfs['VH525628']
df_test = dfs_test['VH525628']

In [702]:
# Include columns that are important (features, labels, student_id)
df_1 = df_1[["student_id", "predict_from", "score_to_predict"]]
df_1['labels'] = df_1['score_to_predict'] - 1
#df_1['predict_from'] = df_1['predict_from'].apply(preprocess)
#df_1['predict_from'] = df_1['predict_from'].apply(contractions.fix)
df_1.head()

<ipython-input-702-b19f5c7ed17b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['labels'] = df_1['score_to_predict'] - 1


,student_id,predict_from,score_to_predict,labels
0,00ONGl5PbU,I believe im right because if you get the bige...,1,0
1,00Pyb7DQQJ,I know this might be right because of the x an...,1,0
2,00mk2jVZeW,Since Z is the smallest and W the biggest you ...,3,2
3,00oZfN72Ey,"i know because x always comes first, then z is...",1,0
4,016tN2x0ht,I know this answer is correct because if you a...,2,1


In [703]:
train_df, test_df = train_test_split(df_1, test_size=0.2, random_state=11, stratify = df_1['labels'])
train_dataset = d1.from_pandas(train_df, preserve_index=True)
test_dataset = d1.from_pandas(test_df, preserve_index=True)

In [688]:
test_df.shape

(3255, 4)

In [704]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=df_1['labels'].nunique())
tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_cases = True)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [705]:
model.resize_token_embeddings(len(tokenizer))

def tokenize_function(examples):
    #examples["predict_from"] = ["[CLS] " + text + " [SEP]" for text in examples["predict_from"]]
    return tokenizer(examples["predict_from"],padding="max_length", truncation=True, 
                     add_special_tokens = True, max_length = 200)



In [706]:
train_datasets = train_dataset.map(tokenize_function, batched=True)
test_datasets = test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/13020 [00:00<?, ? examples/s]

Map:   0%|          | 0/3255 [00:00<?, ? examples/s]

In [648]:
train_datasets, test_datasets

(Dataset({
     features: ['student_id', 'predict_from', 'score_to_predict', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
     num_rows: 13020
 }),
 Dataset({
     features: ['student_id', 'predict_from', 'score_to_predict', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
     num_rows: 3255
 }))

In [708]:
training_args = TrainingArguments(output_dir="./models/model_electra2_prompt_10",
  num_train_epochs=4,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  save_total_limit = 2,
  save_strategy = 'epoch',
  load_best_model_at_end=False
                                  )


trainer = Trainer(model=model,args=training_args,train_dataset=train_datasets,  eval_dataset=test_datasets,
                  compute_metrics=compute_metrics)
trainer.train()

Epoch,Training Loss,Validation Loss,Cohen Kappa
1,No log,0.523278,0.602751
2,0.568300,0.530926,0.637239
3,0.445600,0.547420,0.647493
4,0.324900,0.629998,0.642032


TrainOutput(global_step=1628, training_loss=0.43330372756470625, metrics={'train_runtime': 253.966, 'train_samples_per_second': 205.067, 'train_steps_per_second': 6.41, 'total_flos': 2694931700832000.0, 'train_loss': 0.43330372756470625, 'epoch': 4.0})

In [ ]:
df_results = pd.read_csv('/content/drive/MyDrive/NAEP_Comp/DistillBert/VH525628.csv')

In [ ]:
df_results = df_results.drop(['Unnamed: 0'], axis = 1)
# Rename the column name "_1" to "student_id"
df_results = df_results.rename(columns={"indexes": "student_id", 'True Labels' : 'true_labels', 'PredictedValue': "predicted"})
df_results

,student_id,true_labels,predicted
0,9kVZMPbPk8,1,1
1,oz0idGARrQ,1,1
2,cGhU8gKIBs,3,3
3,dTt8oV8E78,1,1
4,5DDrpdgzne,1,1
...,...,...,...
1565,3fLhSkROx2,1,1
1566,Mg3rOAhX7b,1,1
1567,QHhHl9V1H3,2,1
1568,75rJOuEJMg,1,1


In [709]:
pred, actual, _ = trainer.predict(test_datasets)
pred_labels = np.argmax(pred, axis=1)
cohen_kappa_score(actual, pred_labels, weights='quadratic')

0.6420323325635104

In [710]:
test_indexes = test_dataset['student_id']
score = test_dataset['score_to_predict']

In [714]:
result_VH525628_distill = pd.DataFrame()
result_VH525628_distill['student_id'] = test_indexes
result_VH525628_distill['score'] = score
result_VH525628_distill['true_labels'] = actual +1 
result_VH525628_distill['predicted'] = pred_labels + 1

In [715]:
df_copy = dfs['VH525628'].copy()
df_copy = df_copy[["student_id", "score_to_predict","srace10","dsex","accom2","iep","lep"]]

In [716]:
merged_df = result_VH525628_distill.merge(df_copy, on="student_id", how="inner")
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,7cHQUI8jLe,1,1,1,1,3.0,2.0,2.0,2.0,1.0
1,3nCWMznhIR,1,1,1,1,4.0,1.0,2.0,2.0,2.0
2,KjSbi9amVn,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3,PhQew03NaJ,1,1,1,1,1.0,2.0,2.0,2.0,2.0
4,0psdyfG6bK,1,1,1,1,4.0,1.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...
3250,lHy7ISnHLO,2,2,1,2,2.0,2.0,2.0,2.0,2.0
3251,QmDeYP1yS8,1,1,1,1,1.0,1.0,1.0,1.0,2.0
3252,NMvlO13WU8,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3253,AaBr49YpkU,2,2,3,2,7.0,1.0,2.0,2.0,2.0


In [717]:

merged_df.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/result_VH525628_distill.csv', index = False)
merged_df

,student_id,score,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,7cHQUI8jLe,1,1,1,1,3.0,2.0,2.0,2.0,1.0
1,3nCWMznhIR,1,1,1,1,4.0,1.0,2.0,2.0,2.0
2,KjSbi9amVn,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3,PhQew03NaJ,1,1,1,1,1.0,2.0,2.0,2.0,2.0
4,0psdyfG6bK,1,1,1,1,4.0,1.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...
3250,lHy7ISnHLO,2,2,1,2,2.0,2.0,2.0,2.0,2.0
3251,QmDeYP1yS8,1,1,1,1,1.0,1.0,1.0,1.0,2.0
3252,NMvlO13WU8,1,1,1,1,1.0,1.0,2.0,2.0,2.0
3253,AaBr49YpkU,2,2,3,2,7.0,1.0,2.0,2.0,2.0


In [718]:
true_labels = merged_df['true_labels']
pred = merged_df['predicted']

In [719]:
cohen_kappa_score(pred, true_labels, weights = 'quadratic')

0.6420323325635104

### Test Predictions

In [ ]:
df_test['predict_from'] = df_test['predict_from'].apply(preprocess)
df_test['predict_from'] = df_test['predict_from'].apply(contractions.fix)

In [ ]:
test_dataset = d1.from_pandas(df_test, preserve_index=False)

In [ ]:
test_datasets = test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/1808 [00:00<?, ? examples/s]

In [ ]:
test_datasets

Dataset({
    features: ['student_id', 'accession', 'score_to_predict', 'predict_from', 'year', 'srace10', 'dsex', 'accom2', 'iep', 'lep', 'rater_1', 'pta_rtr1', 'ptb_rtr1', 'ptc_rtr1', 'composite', 'score', 'assigned_score', 'ee_use', 'parsed_xml_v1', 'parsed_xml_v2', 'parsed_xml_v3', 'source1', 'source2', 'source3', 'source4', 'target1', 'target2', 'target3', 'target4', 'eliminations', 'selected', 'eliminated', 'selected1', 'selected2', 'selected3', 'selected4', 'eliminated1', 'eliminated2', 'eliminated3', 'eliminated4', 'selected1.1', 'selected2.1', 'eliminated1.1', 'eliminated2.1', 'partA_response_val', 'partB_response_val', 'partB_eliminations', 'input_ids', 'attention_mask'],
    num_rows: 1808
})

In [ ]:
pred, _, _ = trainer.predict(test_datasets)
pred_labels = np.argmax(pred, axis=1)
pred_labels

array([0, 2, 0, ..., 0, 0, 0])

In [ ]:
df_test_score = df_test.copy()
df_test_score['score_to_predict'] = pred_labels + 1
df_test_score['score_to_predict'].value_counts()

1    1308
2     290
3     210
Name: score_to_predict, dtype: int64

In [ ]:
df_test_score.to_csv('/content/drive/MyDrive/NAEP_Comp/Final Results/test_result_VH525628.csv', index = False)